### Imports

In [1]:
import duckdb
from core.database import duck_to_pandas, expand_column
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from google.cloud import storage
import re

### Events

In [2]:
database_path = '/home/marcelo-borges/Documentos/Projetos/SIAN/laboratory/telemetry-validation/db/telemetria.duckdb'
query = "SELECT * FROM events"
df_events = duck_to_pandas(database_path, query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### EDA

In [3]:
df_events.sample(10)

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,StartPosition,Value,EndDateTime,StartOdometerKilometres,StartDateTime,EventCategory,FuelUsedLitres,EndPosition,EndOdometerKilometres,LocationId
561672,1,1,-3393530750645328945,3306689053612926165,-106647426425259773,1498924535635898368,NaN,0.0,2024-05-24T20:42:21Z,NaN,2024-05-24T20:42:20Z,Detail,0.002138,NaN,NaN,NaN
197679,1,1,7651958308154428287,3293582822612471537,-106647426425259773,1498924246870650880,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.0,2024-04-19T13:03:29Z,918595.10,2024-04-19T13:03:28Z,Detail,NaN,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",918595.10,NaN
166895,1,1,7651958308154428287,3292593541813883077,-106647426425259773,1498924408951140352,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.0,2024-04-16T21:04:34Z,842626.00,2024-04-16T21:04:33Z,Detail,NaN,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",842626.00,NaN
790112,1,1,-3393530750645328945,3314470851857493718,1491699290081755136,1514402039761068032,"{'AgeOfReadingSeconds': 1, 'AltitudeMetres': N...",0.0,2024-06-14T19:59:42Z,619836.70,2024-06-14T19:59:41Z,Detail,0.000500,NaN,NaN,NaN
917885,1,6,-3393530750645328945,3319121828942991053,1491699289142231040,1514472812174508032,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.0,2024-06-27T08:47:57Z,557085.50,2024-06-27T08:47:51Z,Detail,0.004500,NaN,NaN,NaN
234620,1,1,7651958308154428287,3295169202040214287,-106647426425259773,1498925011269795840,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.0,2024-04-23T19:39:27Z,974430.70,2024-04-23T19:39:26Z,Detail,NaN,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",974430.70,NaN
1078338,1,3,-3393530750645328945,3324517272886439121,-5788380741829738886,1498924246870650880,NaN,0.0,2024-07-11T21:44:59Z,NaN,2024-07-11T21:44:56Z,Detail,0.004275,NaN,NaN,NaN
352384,1,1,7651958308154428287,3299306287521376255,-106647426425259773,1498924535635898368,"{'AgeOfReadingSeconds': 2, 'AltitudeMetres': N...",0.0,2024-05-04T23:13:27Z,13201.38,2024-05-04T23:13:26Z,Detail,NaN,"{'AgeOfReadingSeconds': 2, 'AltitudeMetres': N...",13201.38,NaN
1112519,1,1,7651958308154428287,3325818782541179611,1491699286969581568,1498924408951140352,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.0,2024-07-15T09:55:28Z,850548.40,2024-07-15T09:55:27Z,Detail,NaN,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",850548.40,NaN
565177,1,1,-7417774485302453264,3306891330004891555,-106647426425259773,1514472812174508032,"{'AgeOfReadingSeconds': 1, 'AltitudeMetres': N...",NaN,2024-05-25T09:47:17Z,552474.90,2024-05-25T09:47:16Z,Detail,NaN,"{'AgeOfReadingSeconds': 1, 'AltitudeMetres': N...",552474.90,NaN


In [4]:
df_events.describe()

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,Value,StartOdometerKilometres,FuelUsedLitres,EndOdometerKilometres,LocationId
count,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.205826e+06,9.181830e+05,381046.000000,6.378600e+05,1.206700e+04
mean,2.005445e+00,2.073852e+02,1.551935e+18,3.308199e+18,4.865914e+17,1.507137e+18,1.007198e+03,6.964791e+05,0.019021,6.924665e+05,1.508070e+18
std,1.317910e+01,1.397186e+03,5.584598e+18,1.285241e+16,1.217497e+18,1.386682e+16,4.812906e+03,5.094844e+05,0.045152,4.136210e+05,2.815040e+14
min,0.000000e+00,0.000000e+00,-8.078464e+18,3.275198e+18,-9.195140e+18,1.498924e+18,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.507979e+18
25%,1.000000e+00,1.000000e+00,-3.393531e+18,3.297818e+18,-1.066474e+17,1.498925e+18,0.000000e+00,5.489248e+05,0.003741,5.532453e+05,1.507980e+18
50%,1.000000e+00,3.000000e+00,1.443761e+18,3.308728e+18,-1.066474e+17,1.498925e+18,0.000000e+00,6.855360e+05,0.008728,8.499894e+05,1.507981e+18
75%,1.000000e+00,1.500000e+01,7.651958e+18,3.319500e+18,1.491699e+18,1.514417e+18,3.200000e-01,9.167366e+05,0.022444,9.222814e+05,1.507983e+18
max,8.950000e+02,9.269800e+04,8.678978e+18,3.327150e+18,9.206705e+18,1.547012e+18,3.111738e+04,4.050610e+06,5.566263,4.050610e+06,1.508963e+18


In [5]:
df_events.dtypes

TotalOccurances              int64
TotalTimeSeconds             int64
EventTypeId                  int64
EventId                      int64
DriverId                     int64
AssetId                      int64
StartPosition               object
Value                      float64
EndDateTime                 object
StartOdometerKilometres    float64
StartDateTime               object
EventCategory               object
FuelUsedLitres             float64
EndPosition                 object
EndOdometerKilometres      float64
LocationId                 float64
dtype: object

In [6]:
df_events.isna().sum()

TotalOccurances                  0
TotalTimeSeconds                 0
EventTypeId                      0
EventId                          0
DriverId                         0
AssetId                          0
StartPosition               320773
Value                        33129
EndDateTime                  26912
StartOdometerKilometres     320772
StartDateTime                    0
EventCategory                    0
FuelUsedLitres              857909
EndPosition                 601091
EndOdometerKilometres       601095
LocationId                 1226888
dtype: int64

### Uso de memória

#### original

In [21]:
specific_date = '2024-07-18'
# filtered_df = df_positions[df_positions['Timestamp'].dt.date == pd.to_datetime(specific_date).date()]
df_original = df_events[df_events['StartDateTime'].str.contains(specific_date)]

In [22]:
import builtins
memory_usage_bytes = df_original.memory_usage(deep=True)
memory_values = list(memory_usage_bytes)
total_memory_usage_bytes = builtins.sum(memory_values)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_bytes} bytes")
total_memory_usage_mb = total_memory_usage_bytes / (1024 ** 2)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_mb:.2f} MB")
total_memory_usage_gb = total_memory_usage_bytes / (1024 ** 3)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_gb:.2f} GB")

Memória total usada pelo DataFrame: 58951920 bytes
Memória total usada pelo DataFrame: 56.22 MB
Memória total usada pelo DataFrame: 0.05 GB


In [23]:
df_original.to_csv('df_original.csv', sep=';',decimal=',')

#### sugerido

In [24]:
df_sugerido = df_original.copy()

In [25]:
start_position_expanded = expand_column(df_sugerido, 'StartPosition')
end_position_expanded = expand_column(df_sugerido, 'EndPosition')

In [26]:
df_expanded = pd.concat([df_sugerido.drop(columns=['StartPosition', 'EndPosition']), start_position_expanded.add_prefix('StartPosition_'), end_position_expanded.add_prefix('EndPosition_')], axis=1)


In [27]:
df_expanded.describe()

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,Value,StartOdometerKilometres,FuelUsedLitres,EndOdometerKilometres,...,EndPosition_Hdop,EndPosition_Heading,EndPosition_Latitude,EndPosition_Longitude,EndPosition_NumberOfSatellites,EndPosition_OdometerKilometres,EndPosition_Pdop,EndPosition_PositionId,EndPosition_SpeedKilometresPerHour,EndPosition_SpeedLimit
count,60597.000000,60597.000000,6.059700e+04,6.059700e+04,6.059700e+04,6.059700e+04,60597.000000,48376.000000,11575.000000,37080.000000,...,37081.000000,37081.000000,37081.000000,37081.000000,37081.000000,37075.000000,37081.0,3.708100e+04,37081.000000,23495.000000
mean,1.596317,184.207023,7.567854e+17,3.327009e+18,2.443609e+17,1.537603e+18,1445.067649,666404.310789,0.013540,672653.788072,...,1.022842,172.926485,-16.737004,-49.288460,11.987837,672744.503670,0.0,3.327009e+18,22.309876,53.759098
std,9.689379,990.335863,3.880342e+18,9.175486e+13,3.056904e+18,1.806712e+16,3746.143316,234014.461953,0.023141,240205.128034,...,0.737273,103.877560,0.050117,0.044617,0.212571,240094.263288,0.0,9.001194e+13,12.358762,10.899364
min,0.000000,0.000000,-8.078464e+18,3.326779e+18,-8.976664e+18,1.498924e+18,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,-16.976239,-49.454449,5.000000,0.053000,0.0,3.326779e+18,0.000000,30.000000
25%,1.000000,1.000000,-3.393531e+18,3.326938e+18,-1.066474e+17,1.547008e+18,0.000000,580294.675000,0.003500,580331.100000,...,1.000000,79.000000,-16.766032,-49.318716,12.000000,580331.600000,0.0,3.326939e+18,13.663000,40.000000
50%,1.000000,2.000000,1.443761e+18,3.327016e+18,-1.066474e+17,1.547009e+18,800.000000,708414.900000,0.006500,733710.900000,...,1.000000,179.000000,-16.728280,-49.283877,12.000000,733711.100000,0.0,3.327015e+18,23.254000,60.000000
75%,1.000000,13.000000,1.443761e+18,3.327092e+18,3.118870e+17,1.547011e+18,1800.000000,835459.500000,0.016000,835553.425000,...,1.000000,253.000000,-16.702322,-49.258800,12.000000,835553.438000,0.0,3.327090e+18,31.514000,60.000000
max,527.000000,33309.000000,8.678978e+18,3.327150e+18,7.405822e+18,1.547012e+18,29313.470588,982848.200000,0.579000,982848.300000,...,25.000000,359.000000,-16.650273,-49.127992,12.000000,982848.300000,0.0,3.327150e+18,60.727000,110.000000


In [28]:
df_expanded.isnull().sum()

TotalOccurances                             0
TotalTimeSeconds                            0
EventTypeId                                 0
EventId                                     0
DriverId                                    0
AssetId                                     0
Value                                       0
EndDateTime                              1097
StartOdometerKilometres                 12221
StartDateTime                               0
EventCategory                               0
FuelUsedLitres                          49022
EndOdometerKilometres                   23517
LocationId                              60222
StartPosition_AgeOfReadingSeconds       12221
StartPosition_AltitudeMetres            60222
StartPosition_AssetId                   12221
StartPosition_DriverId                  12221
StartPosition_FormattedAddress          12221
StartPosition_Hdop                      12221
StartPosition_Heading                   12221
StartPosition_IsAvl               

In [29]:
df_expanded[df_expanded['EventId']==3289342463373255404]

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,Value,EndDateTime,StartOdometerKilometres,StartDateTime,...,EndPosition_Latitude,EndPosition_Longitude,EndPosition_NumberOfSatellites,EndPosition_OdometerKilometres,EndPosition_Pdop,EndPosition_PositionId,EndPosition_Source,EndPosition_SpeedKilometresPerHour,EndPosition_SpeedLimit,EndPosition_Timestamp


In [30]:
df_expanded['EndDateTime'] = pd.to_datetime(df_expanded['EndDateTime'], errors='coerce')
df_expanded['StartDateTime'] = pd.to_datetime(df_expanded['StartDateTime'], errors='coerce')


In [31]:
float_to_int_columns = [
    'StartPosition_AgeOfReadingSeconds',
    'StartPosition_AssetId',
    'StartPosition_DriverId',
    'StartPosition_NumberOfSatellites',
    'StartPosition_PositionId',
    'EndPosition_AgeOfReadingSeconds',
    'EndPosition_AssetId',
    'EndPosition_DriverId',
    'EndPosition_NumberOfSatellites',
    'EndPosition_PositionId'
]

In [32]:
for column in float_to_int_columns:
    df_expanded[column] = df_expanded[column].astype('Int64', errors='ignore')


In [33]:
data_types_after_casting = df_expanded.dtypes
data_types_after_casting

TotalOccurances                                       int64
TotalTimeSeconds                                      int64
EventTypeId                                           int64
EventId                                               int64
DriverId                                              int64
AssetId                                               int64
Value                                               float64
EndDateTime                             datetime64[ns, UTC]
StartOdometerKilometres                             float64
StartDateTime                           datetime64[ns, UTC]
EventCategory                                        object
FuelUsedLitres                                      float64
EndOdometerKilometres                               float64
LocationId                                          float64
StartPosition_AgeOfReadingSeconds                     Int64
StartPosition_AltitudeMetres                        float64
StartPosition_AssetId                   

In [34]:
import builtins
memory_usage_bytes = df_expanded.memory_usage(deep=True)
memory_values = list(memory_usage_bytes)
total_memory_usage_bytes = builtins.sum(memory_values)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_bytes} bytes")
total_memory_usage_mb = total_memory_usage_bytes / (1024 ** 2)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_mb:.2f} MB")
total_memory_usage_gb = total_memory_usage_bytes / (1024 ** 3)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_gb:.2f} GB")

Memória total usada pelo DataFrame: 56153444 bytes
Memória total usada pelo DataFrame: 53.55 MB
Memória total usada pelo DataFrame: 0.05 GB
